# Surprise 패키지 활용한 추천
빠르고 최적화된 추천 시스텐 활용을 위해 surprise 활용해보자 <br>
설치 !conda install -c conda-forge scikit- <br>
!pip install scikit-surprise

In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

## 기본 실행
<br> 데이터 불러와서 일반적으로 실행

In [2]:
import pandas as pd
from surprise import Reader

ratings = pd.read_csv('./ml-latest-small/ratings.csv')
ratings.to_csv('./ml-latest-small/ratings_noh.csv', index= False, header=False)

서프라이즈로 데이터셋을 불러올때는 인덱스, 헤더를 지운 파일로 저장하고, 리더포멧을 지정하여 읽어온다

In [3]:
reader = Reader(line_format = 'user item rating timestamp', sep= ',', rating_scale = (0.5, 5))
data = Dataset.load_from_file('./ml-latest-small/ratings_noh.csv', reader = reader)

In [4]:
train_set, test_set = train_test_split(data, test_size= 0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(train_set)
pred = algo.test(test_set)
accuracy.rmse(pred)

RMSE: 0.8682


0.8681952927143516

In [5]:
pred[:5] # 예측결과 확ㅣ

[Prediction(uid='63', iid='2000', r_ui=3.0, est=3.5016267817280697, details={'was_impossible': False}),
 Prediction(uid='31', iid='788', r_ui=2.0, est=3.2840758900255937, details={'was_impossible': False}),
 Prediction(uid='159', iid='6373', r_ui=4.0, est=2.804939396068158, details={'was_impossible': False}),
 Prediction(uid='105', iid='81564', r_ui=3.0, est=3.9326180027723914, details={'was_impossible': False}),
 Prediction(uid='394', iid='480', r_ui=3.0, est=3.3135580105479114, details={'was_impossible': False})]

## 추천 알고리즘 클래스
- SVD : 행렬 분해를 통한 잠재요인 협업 필터링
- KNNBasic : 최근접 이웃 협업필터링
- BaselineOnly : 사용자Bias와 아이템Bias를 감안한 SGD베이스라인 알고리즘 <br>
[Surprise_알고리즘 관련 공식문서](http://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html)

> svd 입력 파라미터
- n_factors : 잠재요인 K의 개수, default는 100, 커질수록 정확도는 높아질 수 있으나 과적합 이슈 가능
- n_epochs : SGD(Stochatic Gradient Descent) 수행 시 반복횟수, 디폴트는 20
- biased(bool) : 베이스라인 사용자 편향 적용여부이며, 디폴트는 True <br>
  : 아이템 평가 상 개인의 성향, 즉 편향적 요소를 반영하는 것으로, 베이스라인평점 = 전체평균평점 + 사용자편향점수 + 아이템편향점수

> 베이스라인 평점
- 전체평균평점 = 모든 사용자의 아이템에 대한 평점을 평균한 값
- 사용자편향점수 = 사용자별아이템 평점 평균 값 - 전체평균평점
- 아이템편향점수 = 아이템별 평점 평균 값 - 전체평균평점

예시 : 사용자 A의 어벤저스 베이스라인 평점
- 모든사용자 전체 평균평점 : 3.5
- A사용자 전체영화 평균평점 : 3.0 
- 어벤저스 평균평점 : 4.2 <br>
늘 영화에 대한 판단을 깐깐하게 하는 A사용자의 어벤저스에 대한 베이스라인 평점은<br>
3.5 + (3.0-3.5) + (4.2-3.5) = 3.7

## 교차검증과 하이퍼파라미터 튜닝
- cross_validate()
- GridSearchCV()

In [8]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('./ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5,5))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures = ['MAE','RMSE'], cv=5, verbose = True)

Evaluating MAE, RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.6659  0.6711  0.6706  0.6774  0.6712  0.6713  0.0037  
RMSE (testset)    0.8702  0.8739  0.8733  0.8775  0.8726  0.8735  0.0023  
Fit time          2.52    2.55    2.56    2.53    2.48    2.53    0.03    
Test time         0.05    0.13    0.05    0.05    0.05    0.07    0.03    


{'test_mae': array([0.66591716, 0.67113517, 0.67062096, 0.67742082, 0.67118181]),
 'test_rmse': array([0.87023291, 0.8738905 , 0.87334349, 0.87746443, 0.87257607]),
 'fit_time': (2.517595052719116,
  2.547327995300293,
  2.560128927230835,
  2.534716844558716,
  2.4824540615081787),
 'test_time': (0.04940295219421387,
  0.1275649070739746,
  0.04861307144165039,
  0.048146963119506836,
  0.05282402038574219)}

In [9]:
from surprise.model_selection import GridSearchCV
from surprise import SVD

# 최적화할 파라미터를 딕셔너리 저장
param_grid = {'n_epochs': [20,40,60], 'n_factors':[50,100,200]}

# cv를 3개 폴드세트로 지정, 성능평가는 MAE, RMSE
gs = GridSearchCV(SVD, param_grid, measures = ['rmse','mae'], cv=3)
gs.fit(data)

# 최고 RMSE evaluation 점수와 그때의 하이퍼파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.876711640622501
{'n_epochs': 20, 'n_factors': 50}


## Surprise를 이용한 개인화 영화 추천시스템 구축

In [13]:
# ## 다음 코드는 데이터세트를 분리하지 않고 통채로 넣어서 에러 발생
# data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
# algo = SVD(n_factors=50,random_state=0)
# algo.fit(data)

from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
data_folds = DatasetAutoFolds(ratings_file = './ml-latest-small/ratings_noh.csv', reader=reader) # datasetautofolds 클래스를 ratings_noh.csv파일 기반으로 생성
train_set = data_folds.build_full_trainset()
algo = SVD(n_factors=50,random_state=0)
algo.fit(train_set)


영화속성정보 생성

In [15]:
movies = pd.read_csv('./ml-latest-small/movies.csv')

# userId = 9의 영화데이터 추출
usr_id = 9
mv_id = 42
movieIds = ratings[ratings['userId']==usr_id]['movieId']

if movieIds[movieIds == mv_id].count() == 0:
    print(f'사용자 아이디 {usr_id}는 영화 아이디 {mv_id}의 평점 없음')
print(movies[movies['movieId']==mv_id])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [18]:
uid = str(usr_id)
iid = str(mv_id)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


9번 사용자, 42번 영화에 대한 예측평점은 3.13점. <br>
전체 영화 추출 후 예측평점 높은 순으로 추천해보자

In [19]:
# unseen_movie 추출
def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    total_movies = movies['movieId'].tolist()
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]

    print('평점 매긴 영화수',len(seen_movies),'추천 대상 영화수', len(unseen_movies),
    '전체 영화수',len(total_movies))

    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, usr_id)

평점 매긴 영화수 46 추천 대상 영화수 9696 전체 영화수 9742


In [21]:
# recommend_movie
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    preds = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    def sortkey_est(pred): # 예측 평점 순 정렬
        return pred.est

    preds.sort(key = sortkey_est, reverse= True)
    t1_preds = preds[:top_n]

    # 영화 정보 추출 : 아이디, 제목, 예상평점
    t_ids = [int(pred.iid) for pred in t1_preds]
    t_ratings = [pred.est for pred in t1_preds]
    t_titles = movies[movies.movieId.isin(t_ids)]['title']
    t2_preds = [(id, title, rating) for id, title, rating in zip(t_ids, t_titles, t_ratings)]

    return t2_preds

unseen_movies = get_unseen_surprise(ratings, movies, usr_id)
top_movie_preds = recomm_movie_by_surprise(algo,usr_id,unseen_movies,10)

print()
print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
    print(top_movie[1],":", top_movie[2])


평점 매긴 영화수 46 추천 대상 영화수 9696 전체 영화수 9742

##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122758
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957


In [25]:
preds = [algo.predict(str(usr_id), str(movieId)) for movieId in unseen_movies]
preds

[Prediction(uid='9', iid='1', r_ui=None, est=3.6398015009715396, details={'was_impossible': False}),
 Prediction(uid='9', iid='2', r_ui=None, est=3.0708535569743463, details={'was_impossible': False}),
 Prediction(uid='9', iid='3', r_ui=None, est=2.9912517694217033, details={'was_impossible': False}),
 Prediction(uid='9', iid='4', r_ui=None, est=2.7584914835669805, details={'was_impossible': False}),
 Prediction(uid='9', iid='5', r_ui=None, est=2.653809583804632, details={'was_impossible': False}),
 Prediction(uid='9', iid='6', r_ui=None, est=3.7627640786692917, details={'was_impossible': False}),
 Prediction(uid='9', iid='7', r_ui=None, est=2.883779132627597, details={'was_impossible': False}),
 Prediction(uid='9', iid='8', r_ui=None, est=2.9865774263273677, details={'was_impossible': False}),
 Prediction(uid='9', iid='9', r_ui=None, est=2.7889711267915422, details={'was_impossible': False}),
 Prediction(uid='9', iid='10', r_ui=None, est=3.3040663633820104, details={'was_impossible': 